# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f528bc3abe0>
├── 'a' --> tensor([[ 0.3972, -1.0710,  1.1877],
│                   [-0.7549,  2.0887,  1.3250]])
└── 'x' --> <FastTreeValue 0x7f5288bbe430>
    └── 'c' --> tensor([[-4.6935e-01,  2.4992e-01,  2.5516e+00,  6.6896e-01],
                        [ 1.3332e-01,  2.0062e+00, -4.2402e-01, -5.1131e-01],
                        [-4.8757e-01,  4.1132e-01,  4.9219e-01, -5.4251e-04]])

In [4]:
t.a

tensor([[ 0.3972, -1.0710,  1.1877],
        [-0.7549,  2.0887,  1.3250]])

In [5]:
%timeit t.a

79.9 ns ± 1.2 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f528bc3abe0>
├── 'a' --> tensor([[ 0.1957, -1.1241, -1.7737],
│                   [-0.8629, -2.3149,  0.0979]])
└── 'x' --> <FastTreeValue 0x7f5288bbe430>
    └── 'c' --> tensor([[-4.6935e-01,  2.4992e-01,  2.5516e+00,  6.6896e-01],
                        [ 1.3332e-01,  2.0062e+00, -4.2402e-01, -5.1131e-01],
                        [-4.8757e-01,  4.1132e-01,  4.9219e-01, -5.4251e-04]])

In [7]:
%timeit t.a = new_value

82.9 ns ± 1.71 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.3972, -1.0710,  1.1877],
               [-0.7549,  2.0887,  1.3250]]),
    x: Batch(
           c: tensor([[-4.6935e-01,  2.4992e-01,  2.5516e+00,  6.6896e-01],
                      [ 1.3332e-01,  2.0062e+00, -4.2402e-01, -5.1131e-01],
                      [-4.8757e-01,  4.1132e-01,  4.9219e-01, -5.4251e-04]]),
       ),
)

In [10]:
b.a

tensor([[ 0.3972, -1.0710,  1.1877],
        [-0.7549,  2.0887,  1.3250]])

In [11]:
%timeit b.a

76.6 ns ± 0.706 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-2.4201,  0.1546,  0.4338],
               [ 0.5270,  1.8946, -1.0514]]),
    x: Batch(
           c: tensor([[-4.6935e-01,  2.4992e-01,  2.5516e+00,  6.6896e-01],
                      [ 1.3332e-01,  2.0062e+00, -4.2402e-01, -5.1131e-01],
                      [-4.8757e-01,  4.1132e-01,  4.9219e-01, -5.4251e-04]]),
       ),
)

In [13]:
%timeit b.a = new_value

670 ns ± 14.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.12 µs ± 7.37 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

14.1 µs ± 78.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

268 µs ± 6.56 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

264 µs ± 5.86 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f51bbdacfa0>
├── 'a' --> tensor([[[ 0.3972, -1.0710,  1.1877],
│                    [-0.7549,  2.0887,  1.3250]],
│           
│                   [[ 0.3972, -1.0710,  1.1877],
│                    [-0.7549,  2.0887,  1.3250]],
│           
│                   [[ 0.3972, -1.0710,  1.1877],
│                    [-0.7549,  2.0887,  1.3250]],
│           
│                   [[ 0.3972, -1.0710,  1.1877],
│                    [-0.7549,  2.0887,  1.3250]],
│           
│                   [[ 0.3972, -1.0710,  1.1877],
│                    [-0.7549,  2.0887,  1.3250]],
│           
│                   [[ 0.3972, -1.0710,  1.1877],
│                    [-0.7549,  2.0887,  1.3250]],
│           
│                   [[ 0.3972, -1.0710,  1.1877],
│                    [-0.7549,  2.0887,  1.3250]],
│           
│                   [[ 0.3972, -1.0710,  1.1877],
│                    [-0.7549,  2.0887,  1.3250]]])
└── 'x' --> <FastTreeValue 0x7f51bbdac940>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

50 µs ± 599 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f51bbdacdf0>
├── 'a' --> tensor([[ 0.3972, -1.0710,  1.1877],
│                   [-0.7549,  2.0887,  1.3250],
│                   [ 0.3972, -1.0710,  1.1877],
│                   [-0.7549,  2.0887,  1.3250],
│                   [ 0.3972, -1.0710,  1.1877],
│                   [-0.7549,  2.0887,  1.3250],
│                   [ 0.3972, -1.0710,  1.1877],
│                   [-0.7549,  2.0887,  1.3250],
│                   [ 0.3972, -1.0710,  1.1877],
│                   [-0.7549,  2.0887,  1.3250],
│                   [ 0.3972, -1.0710,  1.1877],
│                   [-0.7549,  2.0887,  1.3250],
│                   [ 0.3972, -1.0710,  1.1877],
│                   [-0.7549,  2.0887,  1.3250],
│                   [ 0.3972, -1.0710,  1.1877],
│                   [-0.7549,  2.0887,  1.3250]])
└── 'x' --> <FastTreeValue 0x7f51bbdac790>
    └── 'c' --> tensor([[-4.6935e-01,  2.4992e-01,  2.5516e+00,  6.6896e-01],
                        [ 1.3332e-01,  2.0062e+00, -4.2402e-01,

In [23]:
%timeit t_cat(trees)

45.6 µs ± 417 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

97.7 µs ± 1.55 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.3972, -1.0710,  1.1877],
                [-0.7549,  2.0887,  1.3250]],
       
               [[ 0.3972, -1.0710,  1.1877],
                [-0.7549,  2.0887,  1.3250]],
       
               [[ 0.3972, -1.0710,  1.1877],
                [-0.7549,  2.0887,  1.3250]],
       
               [[ 0.3972, -1.0710,  1.1877],
                [-0.7549,  2.0887,  1.3250]],
       
               [[ 0.3972, -1.0710,  1.1877],
                [-0.7549,  2.0887,  1.3250]],
       
               [[ 0.3972, -1.0710,  1.1877],
                [-0.7549,  2.0887,  1.3250]],
       
               [[ 0.3972, -1.0710,  1.1877],
                [-0.7549,  2.0887,  1.3250]],
       
               [[ 0.3972, -1.0710,  1.1877],
                [-0.7549,  2.0887,  1.3250]]]),
    x: Batch(
           c: tensor([[[-4.6935e-01,  2.4992e-01,  2.5516e+00,  6.6896e-01],
                       [ 1.3332e-01,  2.0062e+00, -4.2402e-01, -5.1131e-01],
                       [-4.8757e-01,  4

In [26]:
%timeit Batch.stack(batches)

112 µs ± 995 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.3972, -1.0710,  1.1877],
               [-0.7549,  2.0887,  1.3250],
               [ 0.3972, -1.0710,  1.1877],
               [-0.7549,  2.0887,  1.3250],
               [ 0.3972, -1.0710,  1.1877],
               [-0.7549,  2.0887,  1.3250],
               [ 0.3972, -1.0710,  1.1877],
               [-0.7549,  2.0887,  1.3250],
               [ 0.3972, -1.0710,  1.1877],
               [-0.7549,  2.0887,  1.3250],
               [ 0.3972, -1.0710,  1.1877],
               [-0.7549,  2.0887,  1.3250],
               [ 0.3972, -1.0710,  1.1877],
               [-0.7549,  2.0887,  1.3250],
               [ 0.3972, -1.0710,  1.1877],
               [-0.7549,  2.0887,  1.3250]]),
    x: Batch(
           c: tensor([[-4.6935e-01,  2.4992e-01,  2.5516e+00,  6.6896e-01],
                      [ 1.3332e-01,  2.0062e+00, -4.2402e-01, -5.1131e-01],
                      [-4.8757e-01,  4.1132e-01,  4.9219e-01, -5.4251e-04],
                      [-4.6935e-01,  2.4992e-

In [28]:
%timeit Batch.cat(batches)

201 µs ± 3.74 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

635 µs ± 28.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
